# Does any transcript levels correlate with FBA-predicted metabolic fluxes?

## Introduction

Transcriptomics dataset collected from multiple scenarios and published in [Kannan et al 2019](https://doi.org/10.1093/insilicoplants/diz008) was provided from UoI-UC. FBA was used to predict metabolic fluxes under different scenarios and then transcript levels are compared to flux.

## Processing transcript data

In [1]:
import pandas as pd
df = pd.read_csv("/home/sanu/Downloads/gmax_leaf_CO2_normExprn_1.txt",sep="\t")

/home/sanu/.local/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2714: DtypeWarning: Columns (0,2,3,13,14) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [2]:
len(df.columns) -17

229

### Seperating out different studies

Data on light intensity (PPFD) during plant growth was avialable for 5 (of 8) studies. When \[CO2\] is not avilable, ambient levels were assumed. Where day-length is not mentioned, as assumption that daylength is 12h was assumed. In the case of studies involving biotic and abiotic treatments (other than high \[CO2\] treatments), only the WT/control data was used in this analysis.

In [3]:
amb_list = list(df.columns[0:17])
ele_list = list(df.columns[0:17])
for x in df.columns:
    if "Amb" in x:
        amb_list.append(x)
    elif "Ele" in x:
        ele_list.append(x)
    else:
        continue

budRemove_list = list(df.columns[0:17])+["GSM569808.CEL","GSM569809.CEL","GSM569810.CEL",
                                         "GSM569811.CEL","GSM569812.CEL","GSM569813.CEL",
                                         "GSM569814.CEL"]
heatShock_list = list(df.columns[0:17])+["GSM643115.CEL","GSM643116.CEL","GSM643117.CEL","GSM643118.CEL"]
mycInfect_list = list(df.columns[0:17])+["GSM1088939_Schaar_19_wt_c1_S.CEL",
                                         "GSM1088940_Schaar_20_wt_c2_S.CEL",
                                         "GSM1088954_Schaar_25_nts_c1_S.CEL",
                                         "GSM1088955_Schaar_26_nts_c2_S.CEL",
                                         "GSM1088956_Schaar_27_nts_c3_S.CEL"]

In [4]:
df_amb = df[amb_list]
df_ele = df[ele_list]
df_budRemove = df[budRemove_list]
df_heatShock = df[heatShock_list]
df_mycInfect = df[mycInfect_list]

## Set up leaf model and add gene-protein-reaction (GPR) associations from PlantCyc

In [5]:
from cobra import io
from cobra.core import Metabolite
from cobra import flux_analysis
from IPython import display
import logging
logging.basicConfig()
logger = logging.getLogger('logger')

#import sbml file
fname = "/home/sanu/Documents/Scripts/git/plantcoremetabolism-model/PlantCoreMetabolism_v1_3_0.xml"
model = io.sbml.read_sbml_model(fname)
display.clear_output()

from sweetlovegroup.transform import fixModelCompatibilityIssueCobra015
model = fixModelCompatibilityIssueCobra015(model,fname)


In [6]:
len(model.reactions)

892

In [7]:
len(model.metabolites)

861

In [8]:
import pythoncyc
from Functions import *

soy = pythoncyc.select_organism("soy")


In [9]:
model = addGPR2Models(model,soy)

--------------
This list of metabolic reactions are ignored
['RXN_9650_p', '2_KETO_ADIPATE_DEHYDROG_RXN_m', 'Phytol_biosynthesis_p', 'CYSTEINE_AMINOTRANSFERASE_RXN_m', 'GLYCINE_TRNA_LIGASE_RXN_c', 'RXN66_1_c', 'RXN_9648_p', 'RXN-9651', 'Plastidial_ATP_Synthase_p', 'GGPP_biosynthesis_p', 'RXN_9653_p', 'lycopene_biosynthesis_p', 'RXN_2141_p', 'SUCCINYL_COA_HYDROLASE_RXN_m', 'PROTON_ATPase_c', 'MDA_Fd_Ascorbate_p', 'MercaptoPyruvateSulfurtransferase_m', 'Phytol_degradation_p', 'RXN_9652_p', 'A_B_oxidation_x', 'unlProtHYPO_c', 'Mitochondrial_ATP_Synthase_m', 'IPP_biosynthesis_c', 'Mehler_Reaction_p', 'Beta_Oxidation_x', 'HMBPP_synthesis_p', 'OROTATE_REDUCTASE_NADH_RXN_p', 'Ferredoxin_Plastoquinone_Reductase_p', 'RXN_9651_p', 'NADPH_Dehydrogenase_p', 'Plastoquinol_Oxidase_p', 'SUCCINATE_COA_LIGASE_GDP_FORMING_RXN_m', 'RXN_1781_v', 'PREPHENATE_DEHYDROGENASE_NADP_RXN_p', 'PREPHENATEDEHYDROG_RXN_p', 'MALEYLACETOACETATE_ISOMERASE_RXN_c', 'RXN_9654_p', 'LCYSDESULF_RXN_c', 'RXN_9958_NAD_m', 'HEXO

## Modelling fluxes under different scenarios

Create leaf model

In [10]:
########################################################
#This function was used to set up a C3 leaf diel model #
########################################################
def setupC3DielModel(core_model,transferMets,starch_sucrose_ratio=90.0/100):
    from cobra.core import Metabolite, Reaction
    import re

    #create two copies of model elements for day and night
    cobra_model2 = core_model.copy()
    for met in cobra_model2.metabolites:
        met.id = met.id+"1"
        met.compartment = met.compartment+"1"
    for rxn in cobra_model2.reactions:
        rxn.id = rxn.id+"1"

    cobra_model3 = core_model.copy()
    for met in cobra_model3.metabolites:
        met.id = met.id+"2"
        met.compartment = met.compartment+"2"
    for rxn in cobra_model3.reactions:
        rxn.id = rxn.id+"2"

    #merge the day and night model
    cobra_model = cobra_model2+cobra_model3
    for met in cobra_model3.metabolites:
        if not cobra_model.metabolites.__contains__(met.id):
            cobra_model.add_metabolites(met.copy())

    met1 = Metabolite("X_Phloem_contribution_t1",name="Phloem output during the day")
    cobra_model.reactions.get_by_id("Phloem_output_tx1").add_metabolites({met1:1})
    met2 = Metabolite("X_Phloem_contribution_t2",name="Phloem output during at night")
    cobra_model.reactions.get_by_id("Phloem_output_tx2").add_metabolites({met2:1})

    rxn = Reaction("diel_biomass")
    rxn.add_metabolites({met1:-3,met2:-1})
    rxn.lower_bound = 0
    rxn.upper_bound = 1000
    cobra_model.add_reaction(rxn)

    #Adding reactions to allow for day-night metabolite accumulations
    tmfile = open(transferMets,"r")
    tmset=set()
    for line in tmfile:
        tmset.add(line.replace("\n",""))

    for met in tmset:
        if met == "AMMONIUM_v" or met=="FRUCTAN_v":
            continue
        tempRxn = Reaction(met+"_dielTransfer")
        tempRxn.add_metabolites({cobra_model.metabolites.get_by_id(met+"1"):-1,cobra_model.metabolites.get_by_id(met+"2"):1})
        tempRxn.lower_bound=-1000
        if not ((met == "STARCH_p") or (met == "SUCROSE_v") or (met == "MAL_v") or (met == "aMAL_v") or (met == "NITRATE_v") or (met == "CIT_v") or (met == "aCIT_v") or (met == "PROTON_v")):
            tempRxn.lower_bound=0
        tempRxn.upper_bound=1000
        cobra_model.add_reaction(tempRxn)

    fractionMets=dict()
    for rxn in cobra_model.reactions:
        for met in rxn.metabolites.keys():
            prefix=""
            a=re.search("^a{1,3}",met.id)
            anion=""
            if a:
                anion=a.group(0)
                prefix=anion
            b=re.search("^b{1,3}",met.id)
            basic=""
            if b:
                basic=b.group(0)
                prefix=basic
            if ((not prefix == "") and met.compartment == "v1"):
                fractionMets[met]=prefix

    temp=cobra_model.copy()
    for met in fractionMets.keys():
        for rxn in met.reactions:
            if rxn.id.__contains__("_dielTransfer"):
                continue
            else:
                mainMet = met.id[len(fractionMets[met]):]
                coeff1 = temp.reactions.get_by_id(rxn.id).metabolites.get(temp.metabolites.get_by_id(mainMet))
                coeff2 = temp.reactions.get_by_id(rxn.id).metabolites.get(temp.metabolites.get_by_id(met.id))
                if not coeff1:
                    coeff1=0
                if not coeff2:
                    coeff2=0
                total = coeff1 + coeff2
                coeff1 = float(coeff1)/total
                coeff2 = float(coeff2)/total
                if cobra_model.reactions.has_id(met.id[0:len(met.id)-1]+"_dielTransfer"):
                    ub = temp.reactions.get_by_id(met.id[0:len(met.id)-1]+"_dielTransfer").upper_bound
                    lb = temp.reactions.get_by_id(met.id[0:len(met.id)-1]+"_dielTransfer").lower_bound
                    temp.reactions.get_by_id(met.id[0:len(met.id)-1]+"_dielTransfer").remove_from_model()
                    temp.reactions.get_by_id(mainMet[0:len(mainMet)-1]+"_dielTransfer").remove_from_model()
                    Reac = Reaction(mainMet[0:len(mainMet)-1]+"_dielTransfer",name=mainMet+"_dielTransfer")
                    Reac.add_metabolites({temp.metabolites.get_by_id(met.id[0:len(met.id)-1]+"1"):-coeff2,temp.metabolites.get_by_id(met.id[0:len(met.id)-1]+"2"):coeff2,temp.metabolites.get_by_id(mainMet[0:len(mainMet)-1]+"1"):-coeff1,temp.metabolites.get_by_id(mainMet[0:len(mainMet)-1]+"2"):coeff1})
                    Reac.lower_bound=lb
                    Reac.upper_bound=ub
                    temp.add_reaction(Reac)
                    print Reac.reaction
                break
    ####ADD CONSTRAINTS TO MODEL####
    cobra_model = temp.copy()

    #objective function
    cobra_model.reactions.get_by_id("diel_biomass").objective_coefficient=1
    #Leaves - light
    cobra_model.reactions.get_by_id("Sucrose_tx1").lower_bound=0
    cobra_model.reactions.get_by_id("Sucrose_tx1").upper_bound=0
    cobra_model.reactions.get_by_id("GLC_tx1").lower_bound=0
    cobra_model.reactions.get_by_id("GLC_tx1").upper_bound=0
    cobra_model.reactions.get_by_id("CO2_tx1").lower_bound=0
    cobra_model.reactions.get_by_id("NH4_tx1").lower_bound=0
    cobra_model.reactions.get_by_id("NH4_tx1").upper_bound=0
    #Leaves - dark
    cobra_model.reactions.get_by_id("Sucrose_tx2").lower_bound=0
    cobra_model.reactions.get_by_id("Sucrose_tx2").upper_bound=0
    cobra_model.reactions.get_by_id("GLC_tx2").lower_bound=0
    cobra_model.reactions.get_by_id("GLC_tx2").upper_bound=0
    cobra_model.reactions.get_by_id("Photon_tx2").lower_bound=0
    cobra_model.reactions.get_by_id("Photon_tx2").upper_bound=0
    cobra_model.reactions.get_by_id("NH4_tx2").lower_bound=0
    cobra_model.reactions.get_by_id("NH4_tx2").upper_bound=0
    cobra_model.reactions.get_by_id("CO2_tx2").upper_bound=0

    #Set pG6P transporter to 0
    cobra_model.reactions.get_by_id("G6P_Pi_pc1").lower_bound=0
    cobra_model.reactions.get_by_id("G6P_Pi_pc1").upper_bound=0
    cobra_model.reactions.get_by_id("G6P_Pi_pc2").lower_bound=0
    cobra_model.reactions.get_by_id("G6P_Pi_pc2").upper_bound=0

    #Turn off PTOX
    cobra_model.reactions.get_by_id("Plastoquinol_Oxidase_p1").lower_bound=0
    cobra_model.reactions.get_by_id("Plastoquinol_Oxidase_p1").upper_bound=0

    #nitrate uptake constrain
    Nitrate_balance = Metabolite("Nitrate_bal_c", name = "Weights to balance nitrate uptake", compartment = "c1")
    cobra_model.reactions.get_by_id("Nitrate_ec1").add_metabolites({Nitrate_balance:-2})
    cobra_model.reactions.get_by_id("Nitrate_ec2").add_metabolites({Nitrate_balance:3})

    #Rubisco balance
    Rubisco_balance = Metabolite("rubisco_bal_p1", name = "Weights to balance RuBP carboxygenase oxygenase balance", compartment = "p1")
    cobra_model.reactions.get_by_id("RXN_961_p1").add_metabolites({Rubisco_balance:3})
    cobra_model.reactions.get_by_id("RIBULOSE_BISPHOSPHATE_CARBOXYLASE_RXN_p1").add_metabolites({Rubisco_balance:-1})

    #generic ATPase and NADPH oxidase
    Maintenance_constraint = Metabolite("ATPase_NADPHoxidase_constraint_c1",name =  "ATPase_NADPHoxidase_constraint_c1", compartment = "c1")
    Maintenance_constraint2 = Metabolite("ATPase_NADPHoxidase_constraint_c2",name =  "ATPase_NADPHoxidase_constraint_c2", compartment = "c2")
    Maintenance_constraint3 = Metabolite("Light_dark_maintainence_constraint",name =  "Light_dark_maintainence_constraint", compartment = "c1")
    cobra_model.reactions.get_by_id("ATPase_tx1").add_metabolites({Maintenance_constraint:1,Maintenance_constraint3:1})
    cobra_model.reactions.get_by_id("ATPase_tx2").add_metabolites({Maintenance_constraint2:1,Maintenance_constraint3:-1})
    cobra_model.reactions.get_by_id("NADPHoxc_tx1").add_metabolites({Maintenance_constraint:-3})
    cobra_model.reactions.get_by_id("NADPHoxc_tx2").add_metabolites({Maintenance_constraint2:-3})
    cobra_model.reactions.get_by_id("NADPHoxm_tx1").add_metabolites({Maintenance_constraint:-3})
    cobra_model.reactions.get_by_id("NADPHoxm_tx2").add_metabolites({Maintenance_constraint2:-3})
    cobra_model.reactions.get_by_id("NADPHoxp_tx1").add_metabolites({Maintenance_constraint:-3})
    cobra_model.reactions.get_by_id("NADPHoxp_tx2").add_metabolites({Maintenance_constraint2:-3})

    ##constrain sucrose and starch storage
    Sucorse_starch_balance = Metabolite("sucrose_starch_bal_c", name = "Weights to balance sucrose-starch uptake", compartment = "c1")
    cobra_model.reactions.get_by_id("SUCROSE_v_dielTransfer").add_metabolites({Sucorse_starch_balance:-1*starch_sucrose_ratio})
    cobra_model.reactions.get_by_id("STARCH_p_dielTransfer").add_metabolites({Sucorse_starch_balance:1})

    #Plastid enolase was not detected in Arabidopsis mesophyll tissue
    cobra_model.reactions.get_by_id("2PGADEHYDRAT_RXN_p1").lower_bound=0
    cobra_model.reactions.get_by_id("2PGADEHYDRAT_RXN_p1").upper_bound=0
    cobra_model.reactions.get_by_id("2PGADEHYDRAT_RXN_p2").lower_bound=0
    cobra_model.reactions.get_by_id("2PGADEHYDRAT_RXN_p2").upper_bound=0

    #Setting chloroplastic NADPH dehydrogenase to 0  ((Yamamoto et al., 2011)
    cobra_model.reactions.get_by_id("NADPH_Dehydrogenase_p1").lower_bound=0
    cobra_model.reactions.get_by_id("NADPH_Dehydrogenase_p1").upper_bound=0
    cobra_model.reactions.get_by_id("NADPH_Dehydrogenase_p2").lower_bound=0
    cobra_model.reactions.get_by_id("NADPH_Dehydrogenase_p2").upper_bound=0

    #ATP_ADP_Pi constrained to 0 because while there is evidence for its existance, it does not carry flux during the day
    cobra_model.reactions.get_by_id("ATP_ADP_Pi_pc1").lower_bound = 0
    cobra_model.reactions.get_by_id("ATP_ADP_Pi_pc1").upper_bound = 0

    return cobra_model


In [11]:
leaf_model = model.copy()

from sweetlovegroup import transform
leaf_model = setupC3DielModel(leaf_model,"MetabolitesToTransfer.txt",starch_sucrose_ratio=5)

for rxn in leaf_model.reactions:
    if rxn.lower_bound==-1000:
        rxn.lower_bound=-2000
    if rxn.upper_bound==1000:
        rxn.upper_bound=2000
        
leaf_model.reactions.diel_biomass.objective_coefficient=1
sol = flux_analysis.parsimonious.pfba(leaf_model)

0.7 MAL_v1 + 0.3 aMAL_v1 <=> 0.7 MAL_v2 + 0.3 aMAL_v2
0.5 CIT_v1 + 0.5 aCIT_v1 <=> 0.5 CIT_v2 + 0.5 aCIT_v2
bHIS_v1 --> bHIS_v2


### Scenario 1

PPFD = 1500 ([Kannan et al 2019](https://doi.org/10.1093/insilicoplants/diz008))  
\[CO2\] = 380  ([Leakey et al 2009](https://doi.org/10.1073/pnas.0810955106))  
net CO2 uptake rate = 29.85 ([Leakey et al 2009](https://doi.org/10.1073/pnas.0810955106))  
daylength = 14h ([Leakey et al 2009](https://doi.org/10.1073/pnas.0810955106))  


|sampling time|day length|
|-|-|
|11/07/2005|14h50m|
|01/08/2005|14h17m|
|02/08/2005|14h15m|
|23/08/2005|13h28m|
|24/08/2005|13h25m|
|07/09/2005|12h50m|
|18/07/2006|14h42m|
|02/08/2006|14h16m|
|25/08/2006|13h24m|




In [12]:
PPFD = 1500
daylength = 14
ATPase = (0.0049*PPFD) + 2.7851
scale = float(60*60*daylength)/1000000

from sweetlovegroup import analysis
temp = leaf_model.copy()
from Functions import adjustObjectiveBasedOnDaylength
temp = adjustObjectiveBasedOnDaylength(temp,daylength)
temp.reactions.get_by_id("Photon_tx1").lower_bound = 0
temp.reactions.get_by_id("Photon_tx1").upper_bound = PPFD*scale
temp.reactions.get_by_id("ATPase_tx1").lower_bound = 1*scale
temp.reactions.get_by_id("ATPase_tx1").upper_bound = 1*scale
temp.reactions.get_by_id("PYRUVATEORTHOPHOSPHATE_DIKINASE_RXN_p1").upper_bound = 0.034*scale
temp.reactions.get_by_id("OAA_MAL_pc1").upper_bound = 0.75*scale
temp.reactions.get_by_id("OAA_MAL_pc1").lower_bound = -0.75*scale
out_400 = analysis.estimateOutputFromNetCO2(temp,29.85*scale)

leaf_model_400 = leaf_model.copy()
leaf_model_400 = adjustObjectiveBasedOnDaylength(leaf_model_400,daylength)
leaf_model_400.reactions.get_by_id("Photon_tx1").lower_bound = 0
leaf_model_400.reactions.get_by_id("Photon_tx1").upper_bound = PPFD*scale
leaf_model_400.reactions.get_by_id("ATPase_tx1").lower_bound = 1*scale
leaf_model_400.reactions.get_by_id("ATPase_tx1").upper_bound = 1*scale
leaf_model_400.reactions.get_by_id("PYRUVATEORTHOPHOSPHATE_DIKINASE_RXN_p1").upper_bound = 0.034*scale
leaf_model_400.reactions.get_by_id("OAA_MAL_pc1").upper_bound = 0.75*scale
leaf_model_400.reactions.get_by_id("OAA_MAL_pc1").lower_bound = -0.75*scale

leaf_model_400.reactions.diel_biomass.lower_bound = out_400
leaf_model_400.reactions.diel_biomass.upper_bound = out_400
sol = flux_analysis.parsimonious.pfba(leaf_model_400)
print("Phloem output = ")
print(leaf_model_400.reactions.diel_biomass.flux)




75.6
0.0504
1.50444
Phloem output = 
0.0281640962762


In [13]:
import numpy as np


totalTranscript_400 = dict()
for rxn in leaf_model.reactions:
    #print rxn.id
    count = 0
    x = np.zeros((1,len(df_amb.columns[17:])))
    for gene in rxn.genes:
        if gene.id in list(df_amb["Glyma2.0"]):
            #print gene
            if count==0:
                x = df_amb[df_amb["Glyma2.0"]==gene.id][df_amb.columns[17:]].values
                if len(x)>1:
                    for i in range(0,len(x)):
                        if i == 0:
                            x2 = x[i]
                        else:
                            x2 = x2 + x[i]
                    x = np.array([x2])
                count = count+1
            else:
                #print x
                y = df_amb[df_amb["Glyma2.0"]==gene.id][df_amb.columns[17:]].values
                if len(y)>1:
                    for i in range(0,len(y)):
                        if i==0:
                            y2 = y[i]
                        else:
                            y2 = y2 + y[i]
                    y = np.array([y2])
                x = x + y
                count = count+1
                #print x
    #print count
    totalTranscript_400[rxn.id]=x[0]

### Scenario 2

PPFD = 1500 ([Kannan et al 2019](https://doi.org/10.1093/insilicoplants/diz008))    
\[CO2\] = 550 ([Leakey et al 2009](https://doi.org/10.1073/pnas.0810955106))  
net CO2 uptake rate = 36.36 ([Leakey et al 2009](https://doi.org/10.1073/pnas.0810955106))  
daylength = 14h ([Leakey et al 2009](https://doi.org/10.1073/pnas.0810955106))

In [14]:
PPFD = 1500
daylength = 14
ATPase = (0.0049*PPFD) + 2.7851
scale = float(60*60*daylength)/1000000

from sweetlovegroup import analysis
temp = leaf_model.copy()
from Functions import adjustObjectiveBasedOnDaylength
temp = adjustObjectiveBasedOnDaylength(temp,daylength)
temp.reactions.get_by_id("RXN_961_p1").add_metabolites({temp.metabolites.get_by_id("rubisco_bal_p1"):1.35})

temp.reactions.get_by_id("Photon_tx1").lower_bound = 0
temp.reactions.get_by_id("Photon_tx1").upper_bound = PPFD*scale
temp.reactions.get_by_id("ATPase_tx1").lower_bound = 1*scale
temp.reactions.get_by_id("ATPase_tx1").upper_bound = 1*scale
temp.reactions.get_by_id("PYRUVATEORTHOPHOSPHATE_DIKINASE_RXN_p1").upper_bound = 0.034*scale
temp.reactions.get_by_id("OAA_MAL_pc1").upper_bound = 0.75*scale
temp.reactions.get_by_id("OAA_MAL_pc1").lower_bound = -0.75*scale
out_550 = analysis.estimateOutputFromNetCO2(temp,36.36*scale)

leaf_model_550 = leaf_model.copy()
leaf_model_550 = adjustObjectiveBasedOnDaylength(leaf_model_550,daylength)
leaf_model_550.reactions.get_by_id("RXN_961_p1").add_metabolites({leaf_model_550.metabolites.get_by_id("rubisco_bal_p1"):1.35})
leaf_model_550.reactions.get_by_id("Photon_tx1").lower_bound = 0
leaf_model_550.reactions.get_by_id("Photon_tx1").upper_bound = PPFD*scale
leaf_model_550.reactions.get_by_id("ATPase_tx1").lower_bound = 1*scale
leaf_model_550.reactions.get_by_id("ATPase_tx1").upper_bound = 1*scale
leaf_model_550.reactions.get_by_id("PYRUVATEORTHOPHOSPHATE_DIKINASE_RXN_p1").upper_bound = 0.034*scale
leaf_model_550.reactions.get_by_id("OAA_MAL_pc1").upper_bound = 0.75*scale
leaf_model_550.reactions.get_by_id("OAA_MAL_pc1").lower_bound = -0.75*scale
leaf_model_550.reactions.diel_biomass.lower_bound = out_550
leaf_model_550.reactions.diel_biomass.upper_bound = out_550
sol = flux_analysis.parsimonious.pfba(leaf_model_550)
print("Phloem output = ")
print(leaf_model_550.reactions.diel_biomass.flux)




75.6
0.0504
1.832544
Phloem output = 
0.0339353990711


In [15]:
totalTranscript_550 = dict()
temp = df_ele

for rxn in leaf_model.reactions:
    #print rxn.id
    count = 0
    x = np.zeros((1,len(temp.columns[17:])))
    for gene in rxn.genes:
        if gene.id in list(temp["Glyma2.0"]):
            #print gene
            if count==0:
                x = temp[temp["Glyma2.0"]==gene.id][temp.columns[17:]].values
                if len(x)>1:
                    for i in range(0,len(x)):
                        if i == 0:
                            x2 = x[i]
                        else:
                            x2 = x2 + x[i]
                    x = np.array([x2])
                count = count+1
            else:
                #print x
                y = temp[temp["Glyma2.0"]==gene.id][temp.columns[17:]].values
                if len(y)>1:
                    for i in range(0,len(y)):
                        if i==0:
                            y2 = y[i]
                        else:
                            y2 = y2 + y[i]
                    y = np.array([y2])
                x = x + y
                count = count+1
                #print x
    #print count
    totalTranscript_550[rxn.id]=x[0]

### Scenario 3

PPFD = 400 ([Turner et al 2012](https://doi.org/10.1007/s00425-011-1551-4))  
\[CO2\] = 370  (ambient [CO2] assumed)  
net CO2 uptake rate = 23.86 (Predicted according to [Kannan et al 2019](https://doi.org/10.1093/insilicoplants/diz008))  
daylength = 16h ([Turner et al 2012](https://doi.org/10.1007/s00425-011-1551-4))  

In [16]:
PPFD = 400
daylength = 16
ATPase = (0.0049*PPFD) + 2.7851
scale = float(60*60*daylength)/1000000

from sweetlovegroup import analysis
temp = leaf_model.copy()
from Functions import adjustObjectiveBasedOnDaylength
temp = adjustObjectiveBasedOnDaylength(temp,daylength)
temp.reactions.get_by_id("Photon_tx1").lower_bound = 0
temp.reactions.get_by_id("Photon_tx1").upper_bound = PPFD*scale
temp.reactions.get_by_id("ATPase_tx1").lower_bound = 1*scale
temp.reactions.get_by_id("ATPase_tx1").upper_bound = 1*scale
temp.reactions.get_by_id("PYRUVATEORTHOPHOSPHATE_DIKINASE_RXN_p1").upper_bound = 0.034*scale
temp.reactions.get_by_id("OAA_MAL_pc1").upper_bound = 0.75*scale
temp.reactions.get_by_id("OAA_MAL_pc1").lower_bound = -0.75*scale
out_400 = analysis.estimateOutputFromNetCO2(temp,23.86*scale)

leaf_model_budRemove = leaf_model.copy()
leaf_model_budRemove = adjustObjectiveBasedOnDaylength(leaf_model_budRemove,daylength)
leaf_model_budRemove.reactions.get_by_id("Photon_tx1").lower_bound = 0
leaf_model_budRemove.reactions.get_by_id("Photon_tx1").upper_bound = PPFD*scale
leaf_model_budRemove.reactions.get_by_id("ATPase_tx1").lower_bound = 1*scale
leaf_model_budRemove.reactions.get_by_id("ATPase_tx1").upper_bound = 1*scale
leaf_model_budRemove.reactions.get_by_id("PYRUVATEORTHOPHOSPHATE_DIKINASE_RXN_p1").upper_bound = 0.034*scale
leaf_model_budRemove.reactions.get_by_id("OAA_MAL_pc1").upper_bound = 0.75*scale
leaf_model_budRemove.reactions.get_by_id("OAA_MAL_pc1").lower_bound = -0.75*scale
leaf_model_budRemove.reactions.diel_biomass.lower_bound = out_400
leaf_model_budRemove.reactions.diel_biomass.upper_bound = out_400
sol = flux_analysis.parsimonious.pfba(leaf_model_budRemove)
print("Phloem output = ")
print(leaf_model_budRemove.reactions.diel_biomass.flux)




23.04
0.0576
1.374336
Phloem output = 
0.0190870137859


In [17]:
totalTranscript_budRemove = dict()
temp = df_budRemove

for rxn in leaf_model.reactions:
    #print rxn.id
    count = 0
    x = np.zeros((1,len(temp.columns[17:])))
    for gene in rxn.genes:
        if gene.id in list(temp["Glyma2.0"]):
            #print gene
            if count==0:
                x = temp[temp["Glyma2.0"]==gene.id][temp.columns[17:]].values
                if len(x)>1:
                    for i in range(0,len(x)):
                        if i == 0:
                            x2 = x[i]
                        else:
                            x2 = x2 + x[i]
                    x = np.array([x2])
                count = count+1
            else:
                #print x
                y = temp[temp["Glyma2.0"]==gene.id][temp.columns[17:]].values
                if len(y)>1:
                    for i in range(0,len(y)):
                        if i==0:
                            y2 = y[i]
                        else:
                            y2 = y2 + y[i]
                    y = np.array([y2])
                x = x + y
                count = count+1
                #print x
    #print count
    totalTranscript_budRemove[rxn.id]=x[0]

### Scenario 4

PPFD = 350 ([Weston et al 2011](https://doi.org/10.1111/j.1365-3040.2011.02347.x))  
\[CO2\] = 380 ([Weston et al 2011](https://doi.org/10.1111/j.1365-3040.2011.02347.x))  
net CO2 uptake rate = 13.98 (Average of 14.55 and 13.40 \[CO2\]=370) ([Haile nd Higley 2003](https://doi.org/10.1603/0046-225x-32.3.433))  
daylength = 10h ([Weston et al 2011](https://doi.org/10.1111/j.1365-3040.2011.02347.x))  

In [18]:
PPFD = 350
daylength = 10
ATPase = (0.0049*PPFD) + 2.7851
scale = float(60*60*daylength)/1000000

from sweetlovegroup import analysis
temp = leaf_model.copy()
from Functions import adjustObjectiveBasedOnDaylength
temp = adjustObjectiveBasedOnDaylength(temp,daylength)
temp.reactions.get_by_id("Photon_tx1").lower_bound = 0
temp.reactions.get_by_id("Photon_tx1").upper_bound = PPFD*scale
temp.reactions.get_by_id("ATPase_tx1").lower_bound = 1*scale
temp.reactions.get_by_id("ATPase_tx1").upper_bound = 1*scale
temp.reactions.get_by_id("PYRUVATEORTHOPHOSPHATE_DIKINASE_RXN_p1").upper_bound = 0.034*scale
temp.reactions.get_by_id("OAA_MAL_pc1").upper_bound = 0.75*scale
temp.reactions.get_by_id("OAA_MAL_pc1").lower_bound = -0.75*scale
out_370 = analysis.estimateOutputFromNetCO2(temp,13.98*scale)

leaf_model_heatShock = leaf_model.copy()
leaf_model_heatShock = adjustObjectiveBasedOnDaylength(leaf_model_heatShock,daylength)
leaf_model_heatShock.reactions.get_by_id("Photon_tx1").lower_bound = 0
leaf_model_heatShock.reactions.get_by_id("Photon_tx1").upper_bound = PPFD*scale
leaf_model_heatShock.reactions.get_by_id("ATPase_tx1").lower_bound = 1*scale
leaf_model_heatShock.reactions.get_by_id("ATPase_tx1").upper_bound = 1*scale
leaf_model_heatShock.reactions.get_by_id("PYRUVATEORTHOPHOSPHATE_DIKINASE_RXN_p1").upper_bound = 0.034*scale
leaf_model_heatShock.reactions.get_by_id("OAA_MAL_pc1").upper_bound = 0.75*scale
leaf_model_heatShock.reactions.get_by_id("OAA_MAL_pc1").lower_bound = -0.75*scale
leaf_model_heatShock.reactions.diel_biomass.lower_bound = out_370
leaf_model_heatShock.reactions.diel_biomass.upper_bound = out_370
sol = flux_analysis.parsimonious.pfba(leaf_model_heatShock)
print("Phloem output = ")
print(leaf_model_heatShock.reactions.diel_biomass.flux)



12.6
0.036
0.50328
Phloem output = 
0.0153346274356


In [19]:
totalTranscript_heatShock = dict()
temp = df_heatShock

for rxn in leaf_model.reactions:
    #print rxn.id
    count = 0
    x = np.zeros((1,len(temp.columns[17:])))
    for gene in rxn.genes:
        if gene.id in list(temp["Glyma2.0"]):
            #print gene
            if count==0:
                x = temp[temp["Glyma2.0"]==gene.id][temp.columns[17:]].values
                if len(x)>1:
                    for i in range(0,len(x)):
                        if i == 0:
                            x2 = x[i]
                        else:
                            x2 = x2 + x[i]
                    x = np.array([x2])
                count = count+1
            else:
                #print x
                y = temp[temp["Glyma2.0"]==gene.id][temp.columns[17:]].values
                if len(y)>1:
                    for i in range(0,len(y)):
                        if i==0:
                            y2 = y[i]
                        else:
                            y2 = y2 + y[i]
                    y = np.array([y2])
                x = x + y
                count = count+1
                #print x
    #print count
    totalTranscript_heatShock[rxn.id]=x[0]

### Scenario 5

PPFD = 200  
\[CO2\] = ? , assuming 390 based on publication date and CO2 data from the year (https://www.co2.earth/monthly-co2)  
net CO2 uptake rate = 13.73 (Predicted according to [Kannan et al 2019](https://doi.org/10.1093/insilicoplants/diz008) which models [CO2]=370)  
daylength = 16h

In [20]:
PPFD = 200
daylength = 16
ATPase = (0.0049*PPFD) + 2.7851
scale = float(60*60*daylength)/1000000

from sweetlovegroup import analysis
temp = leaf_model.copy()
from Functions import adjustObjectiveBasedOnDaylength
temp = adjustObjectiveBasedOnDaylength(temp,daylength)
temp.reactions.get_by_id("Photon_tx1").lower_bound = 0
temp.reactions.get_by_id("Photon_tx1").upper_bound = PPFD*scale
temp.reactions.get_by_id("ATPase_tx1").lower_bound = 1*scale
temp.reactions.get_by_id("ATPase_tx1").upper_bound = 1*scale
temp.reactions.get_by_id("PYRUVATEORTHOPHOSPHATE_DIKINASE_RXN_p1").upper_bound = 0.034*scale
temp.reactions.get_by_id("OAA_MAL_pc1").upper_bound = 0.75*scale
temp.reactions.get_by_id("OAA_MAL_pc1").lower_bound = -0.75*scale
out_370 = analysis.estimateOutputFromNetCO2(temp,13.73*scale)

leaf_model_mycInfect = leaf_model.copy()
leaf_model_mycInfect = adjustObjectiveBasedOnDaylength(leaf_model_mycInfect,daylength)
leaf_model_mycInfect.reactions.get_by_id("Photon_tx1").lower_bound = 0
leaf_model_mycInfect.reactions.get_by_id("Photon_tx1").upper_bound = PPFD*scale
leaf_model_mycInfect.reactions.get_by_id("ATPase_tx1").lower_bound = 1*scale
leaf_model_mycInfect.reactions.get_by_id("ATPase_tx1").upper_bound = 1*scale
leaf_model_mycInfect.reactions.get_by_id("PYRUVATEORTHOPHOSPHATE_DIKINASE_RXN_p1").upper_bound = 0.034*scale
leaf_model_mycInfect.reactions.get_by_id("OAA_MAL_pc1").upper_bound = 0.75*scale
leaf_model_mycInfect.reactions.get_by_id("OAA_MAL_pc1").lower_bound = -0.75*scale
leaf_model_mycInfect.reactions.diel_biomass.lower_bound = out_370
leaf_model_mycInfect.reactions.diel_biomass.upper_bound = out_370
sol = flux_analysis.parsimonious.pfba(leaf_model_mycInfect)
print("Phloem output = ")
print(leaf_model_mycInfect.reactions.diel_biomass.flux)



11.52
0.0576
0.790848
Phloem output = 
0.0108700826235


In [21]:
0.75*scale

0.0432

In [22]:
leaf_model_mycInfect.reactions.get_by_id("OAA_MAL_pc1").flux/scale

-0.7500000000000001

In [23]:
totalTranscript_mycInfect = dict()
temp = df_mycInfect

for rxn in leaf_model.reactions:
    #print rxn.id
    count = 0
    x = np.zeros((1,len(temp.columns[17:])))
    for gene in rxn.genes:
        if gene.id in list(temp["Glyma2.0"]):
            #print gene
            if count==0:
                x = temp[temp["Glyma2.0"]==gene.id][temp.columns[17:]].values
                if len(x)>1:
                    for i in range(0,len(x)):
                        if i == 0:
                            x2 = x[i]
                        else:
                            x2 = x2 + x[i]
                    x = np.array([x2])
                count = count+1
            else:
                #print x
                y = temp[temp["Glyma2.0"]==gene.id][temp.columns[17:]].values
                if len(y)>1:
                    for i in range(0,len(y)):
                        if i==0:
                            y2 = y[i]
                        else:
                            y2 = y2 + y[i]
                    y = np.array([y2])
                x = x + y
                count = count+1
                #print x
    #print count
    totalTranscript_mycInfect[rxn.id]=x[0]

In [24]:
modelDict = dict()
modelDict[1] = leaf_model_400
modelDict[2] = leaf_model_550
modelDict[3] = leaf_model_budRemove
modelDict[4] = leaf_model_heatShock
modelDict[5] = leaf_model_mycInfect

totalTranscriptDict = dict()
totalTranscriptDict[1] = totalTranscript_400
totalTranscriptDict[2] = totalTranscript_550
totalTranscriptDict[3] = totalTranscript_budRemove
totalTranscriptDict[4] = totalTranscript_heatShock
totalTranscriptDict[5] = totalTranscript_mycInfect

In [56]:
def generateFluxDictIgnoringCompartmentalization(model):
    fluxes = dict()
    for r in model.reactions:
        fluxes[r.id] = r.flux
    fluxes_new = dict()
    processedRxn = set()
    for rxn in fluxes:
        RXN = str(rxn[:rxn.rindex("_")]+rxn[len(rxn)-1]).replace("_NADP","").replace("_NAD","")
        if RXN in  processedRxn:
            continue
        else:
            processedRxn.add(RXN)
        tot = 0
        for rxn2 in fluxes:
            if RXN == str(rxn2[:rxn2.rindex("_")]+rxn2[len(rxn2)-1]).replace("_NADP","").replace("_NAD",""):
                if "RXN_9958" in RXN:
                    print rxn2
                    print abs(fluxes[rxn2])
                    print RXN
                tot = tot+abs(fluxes[rxn2])
        fluxes_new[RXN]=tot
    return fluxes_new
                



modelDictNonCompartmentalized = dict()
for i in range(1,6):
    modelDictNonCompartmentalized[i] = generateFluxDictIgnoringCompartmentalization(modelDict[i])

RXN_9958_NAD_m1
0.0
RXN_99581
RXN_9958_NAD_m2
0.0
RXN_99582
RXN_9958_NAD_m1
0.0
RXN_99581
RXN_9958_NAD_m2
0.0
RXN_99582
RXN_9958_NAD_m1
0.0
RXN_99581
RXN_9958_NAD_m2
0.0
RXN_99582
RXN_9958_NAD_m1
0.0
RXN_99581
RXN_9958_NAD_m2
0.0
RXN_99582
RXN_9958_NAD_m1
0.0
RXN_99581
RXN_9958_NAD_m2
0.0
RXN_99582


In [67]:
modelDictNonCompartmentalized[1]['RXN_99582']

0.0

In [57]:

PCCdict=dict()


In [58]:
from scipy.stats import pearsonr

processedRxn = set()
for rxn in totalTranscriptDict[1].keys():
    R = str(rxn[:rxn.rindex("_")]+rxn[len(rxn)-1]).replace("_NADP","").replace("_NAD","")
    if R in processedRxn:
        continue
    else:
        processedRxn.add(R)
    xlist = list()
    ylist = list()
    for i in range(1,6):
        #print rxn
        for j in range(0,len(totalTranscriptDict[i][rxn])):
            xlist.append(modelDictNonCompartmentalized[i][R])
            ylist.append(totalTranscriptDict[i][rxn][j])
    ans = pearsonr(xlist,ylist)  
    PCCdict[rxn]=ans

In [59]:
PCCdict

{'METHYLCROTONYL_COA_CARBOXYLASE_RXN_m2': (nan, 1.0),
 'METHYLCROTONYL_COA_CARBOXYLASE_RXN_m1': (nan, 1.0),
 'H_mc1': (nan, 1.0),
 'H_mc2': (nan, 1.0),
 'Protein_Processing_c1': (nan, 1.0),
 'Protein_Processing_c2': (nan, 1.0),
 'Mg_biomass2': (nan, 1.0),
 'Mg_biomass1': (nan, 1.0),
 'AMP_pc1': (nan, 1.0),
 'RXN_9958_NAD_m2': (nan, 1.0),
 'RXN_6884_m1': (nan, 1.0),
 'RXN_6884_m2': (nan, 1.0),
 'pTYR_biomass2': (nan, 1.0),
 'pTYR_biomass1': (nan, 1.0),
 'MDA_Fd_Ascorbate_p2': (nan, 1.0),
 'MDA_Fd_Ascorbate_p1': (nan, 1.0),
 '2_PERIOD_3_PERIOD_1_PERIOD_23_RXN_p2': (nan, 1.0),
 '2_PERIOD_3_PERIOD_1_PERIOD_23_RXN_p1': (nan, 1.0),
 'RXN_6902_m1': (nan, 1.0),
 'RXN_6902_m2': (nan, 1.0),
 'pTHR_biomass2': (nan, 1.0),
 'pTHR_biomass1': (nan, 1.0),
 'ASPARAGHYD_RXN_c1': (nan, 1.0),
 'ASPARAGHYD_RXN_c2': (nan, 1.0),
 '2KG_SUC_mc1': (nan, 1.0),
 'HIS_ORNITHINE_mc2': (nan, 1.0),
 'HIS_ORNITHINE_mc1': (nan, 1.0),
 'RXN_11832_m1': (nan, 1.0),
 'RXN_11832_m2': (nan, 1.0),
 'GLUTDECARBOX_RXN_c1': (-0.

In [68]:
fout = open("Data/PPC_transcripts_fluxes_lightNotForced_constMaint_notAverage_ignoreComp_27_12_19.tsv","w")

fout.write("Rxn ID\treaction\tPCC\tp-value\tfluxes\n")
for rxn in PCCdict.keys():
    xlist = list()
    ylist = list()
    for i in range(1,6):
        xlist.append(modelDictNonCompartmentalized[i][str(rxn[:rxn.rindex("_")]+rxn[len(rxn)-1]).replace("_NADP","").replace("_NAD","")])
        ylist.append(totalTranscriptDict[i][rxn])
    fout.write(rxn+"\t"+leaf_model.reactions.get_by_id(rxn).reaction+"\t"+str(PCCdict[rxn][0])+"\t"+str(PCCdict[rxn][1])+"\t"+str(xlist)+"\n")
fout.close()

In [76]:
rxn = leaf_model.reactions.get_by_id(rxn)
rxn.genes

frozenset({<Gene Glyma.14G009600 at 0x7fa1d6394550>,
           <Gene Glyma.02G304200 at 0x7fa1d6394590>,
           <Gene Glyma.04G173400 at 0x7fa1d63945d0>,
           <Gene Glyma.04G144300 at 0x7fa1d6394610>,
           <Gene Glyma.19G068500 at 0x7fa1d6394650>,
           <Gene Glyma.04G157000 at 0x7fa1d6394690>,
           <Gene Glyma.03G190500 at 0x7fa1d63946d0>,
           <Gene Glyma.19G190900 at 0x7fa1d6394710>,
           <Gene Glyma.15G235300 at 0x7fa1d6394750>,
           <Gene Glyma.06G208200 at 0x7fa1d6394790>,
           <Gene Glyma.16G204600 at 0x7fa1d63947d0>,
           <Gene Glyma.09G153900 at 0x7fa1d6394810>,
           <Gene Glyma.15G207600 at 0x7fa1d6394850>})

### Plot fluxes which show +ve correlation with transcript levels

In [71]:
rxnlist = ["2PGADEHYDRAT_RXN_c1"]

colordict = ["white","black","blue","green","red","cyan"]

for rxn in rxnlist:
    max_x = 0
    min_x = 0
    for i in range(1,6):
        xlist = list()
        ylist = list()
        for j in range(0,len(totalTranscriptDict[i][rxn])):
            xlist.append(modelDictNonCompartmentalized[i][rxn[:rxn.rindex("_")]+rxn[len(rxn)-1].replace("NADP","").replace("NAD","")])
            ylist.append(totalTranscriptDict[i][rxn][j])
        import matplotlib.pyplot as plt
        plt.scatter(xlist,ylist,color=colordict[i],label="Scenario "+str(i))
        if max_x<max(xlist):
            max_x = max(xlist)
        if min_x>min(xlist):
            min_x = max(xlist)
    plt.title(rxn)
    plt.xlabel("predicted flux")
    plt.ylabel("transcript abundance")
    plt.legend()
    if abs(min_x) > abs(max_x):
        temp = min_x
    else:
        temp = max_x
    plt.xlim(min_x-(temp*(10.0/100)),max_x+(temp*(10.0/100)))
    plt.savefig("Data/Plots_27_12_2019/"+rxn+".svg")
    plt.close()
        

## References
- Kannan, K., Wang, Y., Lang, M., Challa, G. S., Long, S. P., & Marshall-Colon, A. (2019). Combining gene network, metabolic and leaf-level models shows means to future-proof soybean photosynthesis under rising CO2. In Silico Plants, 1(1), 1–18. https://doi.org/10.1093/insilicoplants/diz008
- Haile, F. J., & Higley, L. G. (2003). Changes in Soybean Gas-Exchange After Moisture Stress and Spider Mite Injury. Environmental Entomology, 32(3), 433–440. https://doi.org/10.1603/0046-225x-32.3.433
- Turner, G. W., Cuthbertson, D. J., Voo, S. S., Settles, M. L., Grimes, H. D., & Lange, B. M. (2012). Experimental sink removal induces stress responses, including shifts in amino acid and phenylpropanoid metabolism, in soybean leaves. Planta, 235(5), 939–954. https://doi.org/10.1007/s00425-011-1551-4
- Leakey, A. D. B., Xu, F., Gillespie, K. M., McGrath, J. M., Ainsworth, E. A., & Ort, D. R. (2009). Genomic basis for stimulated respiration by plants growing under elevated carbon dioxide. Proceedings of the National Academy of Sciences of the United States of America, 106(9), 3597–3602. https://doi.org/10.1073/pnas.0810955106
- Weston, D. J., Karve, A. A., Gunter, L. E., Jawdy, S. S., Yang, X., Allen, S. M., & Wullschleger, S. D. (2011). Comparative physiology and transcriptional networks underlying the heat shock response in Populus trichocarpa, Arabidopsis thaliana and Glycine max. Plant, Cell and Environment, 34(9), 1488–1506. https://doi.org/10.1111/j.1365-3040.2011.02347.x
- Bunce, J. (2019). Consistent Differences in Field Leaf Water-Use Efficiency among Soybean Cultivars. Plants, 8(5), 123. https://doi.org/10.3390/plants8050123